# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

дополнения: *Ян Пиле, НИУ ВШЭ*


### Автоматизация работы в браузере: библиотека `selenium`

Библиотека `selenium` – набор инструментов для интерактивной работы в браузере средствами Python. Вообще Selenium ‒ это целый проект, в котором есть разные инструменты. Мы рассмотрим один из самых распространенных ‒ Selenium WebDriver, модуль, который позволяется Python встраиваться в браузер и работать в нем как пользователь: кликать на ссылки и кнопки, заполнять формы, выбирать опции в меню и прочее. 

Мы будем использовать WebDriver для решения такой задачи. Необходимо выгрузить все адреса участковых избирательных  комиссий Ивановской области. Для этого нужно написать код, который будет открывать в окне браузера раздел *По номеру избирательного участка*, вводить в поле с номером номер участка и выбирать регион из предлагаемого списка. Итак, начнем.

Сначала загрузим веб-драйвер из библиотеки `selenium`. 

In [74]:
from selenium import webdriver as wb
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# browser = webdriver.Firefox()

Если Python пишет `No module called selenium`, убедитесь, что у вас установлена эта библиотека. Самый надежный способ установить ее ‒ найти *Anaconda Command Prompt*, вписать строку `pip install selenium` и нажать *Enter*. Если *Anaconda Command Prompt* не находится, можно поступить так: запустить Jupyter Notebook, щелкнуть на черное окно консоли, нажать *Ctrl+Z* (остановить запуск Jupyter), а потом так же ввести в этом окне строку `pip install selenium` и нажать *Enter*.

Затем нужно выбрать браузер и открыть новое окно через Python. Для этого нужно вызвать функцию, которая отвечает за открытие браузера. Обычно используется Chrome.

Так случилось, что мне использовать Chrome нельзя, поэтому я все буду показывать на примере Firefox. 

In [6]:
import os
os.getcwd()

'/Users/ianpile/DPO 2020'

In [66]:
from selenium import webdriver

driver = webdriver.Firefox()

Если код выше не исполняется, скачайте файл с веб-драйвером [отсюда](https://sites.google.com/a/chromium.org/chromedriver/downloads), распакуйте архив и пропишите путь к файлу в круглых скобках (в примере файл с расширением exe на Windows). 

In [ ]:
# br = wb.Chrome('C:/Users/student/Desktop/chromewebdriver/chromedriver.exe')

In [2]:
# br = wb.Chrome("/Users/allat/Downloads/chromedriver")

In [67]:
driver.get("http://www.cikrf.ru/services/lk_address/?do=find_by_uik")

Ура, страница открылась. Что на этой странице есть интересного? Два поля: ввод номера участка и регион. Сохраним номер участка в переменную `n_uik`, а регион ‒ в `reg`.

In [68]:
n_uik = 244
reg = "Ивановская область"

Вопрос: как эти два поля заполнить? Нужно найти их на странице, открытой в браузере, и вписать туда нужные строки. Только сделать это нужно через Python. Воспользуемся инструментом CSS Selector (установить расширение для Chrome можно [здесь](https://chrome.google.com/webstore/detail/copy-css-selector/kemkenbgbgodoglfkkejbdcpojnodnkg)). Для этого нужно открыть страницу в обычном браузере и кликнуть на расширение в правом углу. 

![](css.png)

Теперь, когда мы будем наводить курсор мыши на объект на странице в таком режиме и кликать, внизу будет отображаться его название в css. 

![](css-uik.png)

Теперь осталось зафиксировать поле с таким названием и ввести туда номер УИКа. 

In [80]:
# находим поле с #uik и сохраняем
uik_field = driver.find_element_by_css_selector("#uik")

In [81]:
# вводим номер УИКа в поле - метод send_keys
uik_field.send_keys(n_uik)

![](css-enter.png)

Ура, получилось. А как быть с регионом? Там же не поле ввода, а целое выпадающее меню с опциями... Попробуем действовать аналогичным образом.

In [76]:
region_field = driver.find_element_by_name("subject")
region_field.send_keys(reg)

Почему-то вместо Ивановской области выпала Тамбовская... Но ведь у нас здесь не просто поле для заполнения текстом, а выпадающее меню. У Web-driver'a есть специальный инструмент работы с такими полями - Селектор.

In [77]:
# region_field - поле для выбора региона, нашли по названию
select = Select(driver.find_element_by_name("subject"))
select.select_by_visible_text(reg)

Сработало! но можно выбирать еще и по каким-то свойстуам элементов из селектора.

In [ ]:
select.select_by_value('37') 

Осталось только кликнуть на кнопку *Отправить запрос*. Сначала найдем ее с помощью CSS Selector, а потом кликнем по ней ‒ воспользуемся методом `.click()`:

In [82]:
button = driver.find_element_by_link_text("Отправить запрос")
button.click()

В браузере открылась страница с адресом избирательного участка. 

![](page.png)

Осталось подгрузить `re` и найти на странице адрес участка с помощью регулярных выражений.

In [83]:
from time import sleep
driver.get("http://www.cikrf.ru/services/lk_address/?do=find_by_uik")
uik_field = driver.find_element_by_css_selector("#uik")
uik_field.send_keys(n_uik)
    
select = Select(driver.find_element_by_name("subject"))
select.select_by_visible_text(reg)
sleep(1.5) # еще добавим задержку в 1.5 секунды
button = driver.find_element_by_link_text("Отправить запрос")
button.click()
sleep(1.5) # еще добавим задержку в 1.5 секунды

In [85]:
import re

In [86]:
p = re.search(r"Адрес помещения для голосования: ([^<]+)", driver.page_source)
p

<re.Match object; span=(1741, 1919), match='Адрес помещения для голосования: 155800, Ивановск>

In [87]:
p.group(1) # текст адреса

'155800, Ивановская область, городской округ Кинешма, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"'

Получилось! Единственное, хорошо бы учесть случаи, когда адреса участка в таком виде на странице нет (такие случаи бывают: иногда страница создана не по шаблону, иногда указан адрес территориальной комиссии). Для этого нам понадобится условие. Добавим «развилку»: пусть Python пробует найти адрес через указанное регулярное выражение, а если не найдет, то ищет его с помощью другого регулярного выражения. 

In [88]:
if p is None:
    p = re.search(r"Адрес: ([^<]+)", br.page_source)
    addr = p.group(1)

Теперь у нас есть универсальный код, который позволяет найти адрес избирательного участка по номеру. В следующий раз мы оформим этот код в функцию, чтобы можно было подставлять в нее любой номер и регион, и применять ее в цикле, итерируя по номерам участков.